# 目录
1. LSTM原理及理解
2. 代码实现
2.1 库的介绍
2.2 数据介绍
2.3 数据标准化
2.4 运用LSTM神经网络
3. 归纳总结

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'

In [5]:
# 读入数据文件
df = pd.read_csv('./input/ts_000001_2017-03-01_2019-04-19.csv', encoding='gbk')

FileNotFoundError: [Errno 2] File b'./input/ts_000001_2017-03-01_2019-04-19.csv' does not exist: b'./input/ts_000001_2017-03-01_2019-04-19.csv'

In [6]:
# 展示数据
df.head() # 显示DataFrame的前若干行，默认为5

NameError: name 'df' is not defined

In [7]:
# 取出数据中需要的部分，从第二列取至最后一列
data = df.iloc[:,1:].values

NameError: name 'df' is not defined

In [8]:
# 展示数据
df.iloc[:,1:].head()

NameError: name 'df' is not defined

In [9]:
df_demo = df.iloc[:,2:].head()
df_demo = (df_demo - np.mean(df_demo,axis=0))/np.std(df_demo,axis=0)
df_demo

NameError: name 'df' is not defined

In [10]:
#定义常量
rnn_unit=10        #hidden layer units
input_size=13      # 输入层维度
output_size=1      # 输出层维度
lr=0.0006          # 学习率
#输入层、输出层权重、偏置
weights={
         'in':tf.Variable(tf.random_normal([input_size,rnn_unit])),
         'out':tf.Variable(tf.random_normal([rnn_unit,1]))
         }
biases={
        'in':tf.Variable(tf.constant(0.1,shape=[rnn_unit,])),
        'out':tf.Variable(tf.constant(0.1,shape=[1,]))
        }

NameError: name 'tf' is not defined

In [11]:
# 获取训练集
def get_train_data(batch_size,time_step,train_begin,train_end):
    batch_index=[]
    data_train=data[train_begin:train_end]
    normalized_train_data=(data_train-np.mean(data_train,axis=0))/np.std(data_train,axis=0)  #标准化
    train_x,train_y=[],[]   #训练集x和y
    for i in range(len(normalized_train_data)-time_step):
        if i % batch_size==0:
            batch_index.append(i)
        x=normalized_train_data[i:i+time_step,:13]
        y=normalized_train_data[i:i+time_step,13,np.newaxis]
        train_x.append(x.tolist())
        train_y.append(y.tolist())
    batch_index.append((len(normalized_train_data)-time_step))
    return batch_index,train_x,train_y

In [12]:
# 获取测试集
def get_test_data(time_step,test_begin):
    data_test=data[test_begin:]
    mean, std = np.mean(data_test, axis=0), np.std(data_test, axis=0)
    normalized_test_data=(data_test-np.mean(data_test,axis=0))/np.std(data_test,axis=0)  #标准化
    size=(len(normalized_test_data)+time_step-1)//time_step
    test_x,test_y=[],[]   #测试集x和y
    for i in range(size-1):
        x=normalized_test_data[i*time_step:(i+1)*time_step,:13]
        y=normalized_test_data[i*time_step:(i+1)*time_step,13]
        test_x.append(x.tolist())
        test_y.extend(y)
    test_x.append((normalized_test_data[(i+1)*time_step:,:13]).tolist())
    test_y.extend((normalized_test_data[(i+1)*time_step:,13]).tolist())
    return test_x,test_y, mean, std

In [13]:
# 构建神经网络
def lstm(X):     
    batch_size=tf.shape(X)[0]
    time_step=tf.shape(X)[1]
    w_in=weights['in']
    b_in=biases['in']  
    inputdata=tf.reshape(X,[-1,input_size])  #需要将tensor转成2维进行计算，计算后的结果作为隐藏层的输入
    input_rnn=tf.matmul(inputdata,w_in)+b_in
    input_rnn=tf.reshape(input_rnn,[-1,time_step,rnn_unit])  #将tensor转成3维，作为lstm cell的输入
    cell=tf.nn.rnn_cell.BasicLSTMCell(rnn_unit)
    init_state=cell.zero_state(batch_size,dtype=tf.float32)
    output_rnn,final_states=tf.nn.dynamic_rnn(cell, input_rnn,initial_state=init_state, dtype=tf.float32)  #output_rnn是记录lstm每个输出节点的结果，final_states是最后一个cell的结果
    output=tf.reshape(output_rnn,[-1,rnn_unit]) #作为输出层的输入
    w_out=weights['out']
    b_out=biases['out']
    pred=tf.matmul(output,w_out)+b_out
    return pred,final_states

In [14]:
# 训练模型
def train_lstm(batch_size,time_step,train_begin,train_end,n):
    X = tf.placeholder(tf.float32, shape=[None, time_step, input_size])
    Y = tf.placeholder(tf.float32, shape=[None, time_step, output_size])
    # 获得训练集的数据及其标签和batch_size
    batch_index,train_x,train_y=get_train_data(batch_size,time_step,train_begin,train_end)
    pred,_ = lstm(X)
    # 损失函数
    loss = tf.reduce_mean(tf.square(tf.reshape(pred,[-1])-tf.reshape(Y, [-1])))    
    # 创建Adam优化器，最小化loss
    train_op = tf.train.AdamOptimizer(lr).minimize(loss)    
    # 创建一个Saver，设置需要保留的最近检查点文件最大数为15，用以参数恢复
    saver = tf.train.Saver(tf.global_variables(),max_to_keep=15)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        # 重复训练n次
        for i in range(n):
            for step in range(len(batch_index)-1):
                _,loss_=sess.run([train_op,loss],feed_dict={X:train_x[batch_index[step]:batch_index[step+1]],Y:train_y[batch_index[step]:batch_index[step+1]]})
            # 进度每到10%输出一次情况，并且保存一次参数
            if (i+1) % (n/10)==0:
                print(i+1,loss_, "保存模型",)
                saver.save(sess, './model_stock/stock.model')

In [15]:
# 预测模型
def prediction(time_step,test_begin):
    X=tf.placeholder(tf.float32, shape=[None,time_step,input_size])
    test_x,test_y, mean, std=get_test_data(time_step, test_begin)
    pred,_=lstm(X)     
    saver=tf.train.Saver(tf.global_variables())
    with tf.Session() as sess:
        # 参数恢复
        module_file = tf.train.latest_checkpoint('./model_stock')   
        saver.restore(sess, module_file)   #取训练集最后一次保存数据
        test_predict=[]
        # 从测试集中通过循环的方式取出其特征维度，放入模型进行预测，并且记下预测结果
        for step in range(len(test_x)-1):
            prob=sess.run(pred,feed_dict={X:[test_x[step]]})   
            predict=prob.reshape((-1))
            test_predict.extend(predict)
        # 将标准化的数据还原至真实数据值，并且计算测试集偏差
        test_y=np.array(test_y)*std[13]+mean[13]
        test_predict=np.array(test_predict)*std[13]+mean[13]
        # acc为测试集偏差
        acc=np.average(np.abs(test_predict-test_y[:len(test_predict)])/test_y[:len(test_predict)]) 
        print(acc)
        # 折线图展示
        plt.figure(figsize=(20,6))
        data_list=['04-22', '04-23', '04-24', '04-25', '04-26', 
                   '04-29', '04-30', '05-01', '05-02', '05-03',
                   '05-06', '05-07', '05-08', '05-09', '05-10',
                   '05-11', '05-12', '05-13', '05-14', '05-15']
        plt.plot(data_list, test_y[0:20], color='r')
        plt.plot(data_list, test_predict[0:20], color='b')
        plt.xlabel('date')
        plt.ylabel('stock index')
        #plt.plot(list(range(len(test_predict))), test_predict, color='b')
        #plt.plot(list(range(len(test_y))), test_y, color='r')
        plt.show()

In [16]:
# 设置模型训练预测需要的量
batch_size = 60
time_step = 20
train_begin = 0
train_end = 500
test_begin = 500
n = 10000

In [17]:
# 训练模型
with tf.variable_scope('train', reuse=tf.AUTO_REUSE):
    train_lstm(batch_size,time_step,train_begin,train_end, n)

NameError: name 'tf' is not defined

In [18]:
# 模型预测
with tf.variable_scope('train', reuse=tf.AUTO_REUSE):
    prediction(time_step,test_begin)

NameError: name 'tf' is not defined